In [66]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.eval.med_status
import datasets
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
results_dir = '../results/'
# results_dir = '../results_tmp/'
from clin.config import PATH_REPO
import imodelsx.process_results
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[[col for col in r.columns if not col in ['checkpoint_verify', 'role_verify']]]

def viz_blues(df):
    return df.style.format(precision=3).background_gradient(cmap='Blues')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 106/106 [00:00<00:00, 1889.15it/s]


In [67]:
# medication_status
r_med = r[r.dataset_name == 'medication_status']
df = pd.DataFrame.from_dict(datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')['test'])
r_med = clin.eval.med_status.add_status_eval(r_med, df)

  0%|          | 0/2 [00:00<?, ?it/s]

/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

# Average results

In [68]:
# medication_status
r_med = r[r.dataset_name == 'medication_status']
df = pd.DataFrame.from_dict(datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')['test'])
r_med = clin.eval.med_status.add_status_eval(r_med, df)
r_med = imodelsx.process_results.average_over_seeds(r_med, experiment_filename=join(PATH_REPO, 'experiments', 'eval_model.py'))

# compare values for a single row
row_df = r_med[(r_med.n_shots == 5) * (r_med.checkpoint == 'text-davinci-003')].reset_index()
rc = row_df[[c for c in row_df.columns if '___' in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split('___')) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = rc.rename(columns={
    'level_0': '',
    'level_1': 'Verifiers',
}).pivot_table(index='Verifiers', columns='', values=0).round(3)
# rc.style.format(precision=3).background_gradient(cmap='gray')

cols = {
    'f1': 'F1 (Med)',
    'precision': 'Precision (Med)',
    'recall': 'Recall (Med)',
    'status_f1_macro_cond': 'F1 (Med status)',
}
rows = {
    'original': 'Original',
    'ov': 'Omission',
    'pv': 'Prune',
    'ov_pv': 'Omission + Prune',
    'sv': 'Omission + Prune + Evidence'
}
rt_med = rc[list(cols.keys())].rename(columns=cols).loc[list(rows.keys())].rename(index=rows)
viz_blues(rt_med)

  0%|          | 0/2 [00:00<?, ?it/s]

gr ['dataset_name', 'save_dir', 'checkpoint', 'n_shots']
num ['seed', 'use_cache', 'recall___original', 'precision___original', 'f1___original', 'recall___ov', 'precision___ov', 'f1___ov', 'recall___pv', 'precision___pv', 'f1___pv', 'recall___ev', 'precision___ev', 'f1___ev', 'recall___ov_pv', 'precision___ov_pv', 'f1___ov_pv', 'recall___ov_pv_ev', 'precision___ov_pv_ev', 'f1___ov_pv_ev', 'recall___sv', 'precision___sv', 'f1___sv', 'status_acc_cond___original', 'status_f1_macro_cond___original', 'status_acc_cond___ov', 'status_f1_macro_cond___ov', 'status_acc_cond___pv', 'status_f1_macro_cond___pv', 'status_acc_cond___ev', 'status_f1_macro_cond___ev', 'status_acc_cond___ov_pv', 'status_f1_macro_cond___ov_pv', 'status_acc_cond___ov_pv_ev', 'status_f1_macro_cond___ov_pv_ev', 'status_acc_cond___sv', 'status_f1_macro_cond___sv']


/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,F1 (Med),Precision (Med),Recall (Med),F1 (Med status)
Verifiers,,,,
Original,0.929,0.929,0.928,0.561
Omission,0.913,0.881,0.946,0.561
Prune,0.932,0.949,0.916,0.561
Omission + Prune,0.935,0.940,0.931,0.561
Omission + Prune + Evidence,0.935,0.942,0.928,0.581


In [69]:
r_ebm = r[r.dataset_name == "ebm"]
row = r_ebm[(r_ebm.n_shots == 5) * (r_ebm.checkpoint == "text-davinci-003")].iloc[0]

# show metrics
row_df = pd.DataFrame(
    pd.Series({k: row[k] for k in row.keys() if "___" in k}).round(3)
).T
rc = row_df[[c for c in row_df.columns if "___" in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split("___")) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = (
    rc.rename(
        columns={
            "level_0": "",
            "level_1": "Verifiers",
        }
    )
    .pivot_table(index="Verifiers", columns="", values=0)
    .round(3)
)
# rc.index = [x.replace("list_", "") for x in rc.index.values]
cols = {
    "f1": "F1 (Arms)",
    "precision": "Precision (Arms)",
    "recall": "Recall (Arms)",
}
rows = {
    "original": "Original",
    "ov": "Omission",
    "pv": "Prune",
    "ov_pv": "Omission + Prune",
    "ov_pv_ev": "Omission + Prune + Evidence",
}
rt_ebm = rc[list(cols.keys())].rename(columns=cols).loc[list(rows.keys())].rename(index=rows)
viz_blues(rt_ebm)

,F1 (Arms),Precision (Arms),Recall (Arms)
Verifiers,,,
Original,0.506,0.489,0.523
Omission,0.505,0.442,0.589
Prune,0.580,0.589,0.570
Omission + Prune,0.574,0.530,0.626
Omission + Prune + Evidence,0.590,0.570,0.612


# Visualize

In [72]:
# add columns from rt_ebm to rt_med
rt = rt_med.join(rt_ebm) #, rsuffix=' (Arms)')
rt = rt.drop(columns='F1 (Med status)')
display(viz_blues(rt))

,F1 (Med),Precision (Med),Recall (Med),F1 (Arms),Precision (Arms),Recall (Arms)
Verifiers,,,,,,
Original,0.929,0.929,0.928,0.506,0.489,0.523
Omission,0.913,0.881,0.946,0.505,0.442,0.589
Prune,0.932,0.949,0.916,0.580,0.589,0.570
Omission + Prune,0.935,0.940,0.931,0.574,0.530,0.626
Omission + Prune + Evidence,0.935,0.942,0.928,0.590,0.570,0.612


In [76]:
print(rt.style.format(precision=3).to_latex(hrules=True))

\begin{tabular}{lrrrrrr}
\toprule
 & F1 (Med) & Precision (Med) & Recall (Med) & F1 (Arms) & Precision (Arms) & Recall (Arms) \\
Verifiers &  &  &  &  &  &  \\
\midrule
Original & 0.929 & 0.929 & 0.928 & 0.506 & 0.489 & 0.523 \\
Omission & 0.913 & 0.881 & 0.946 & 0.505 & 0.442 & 0.589 \\
Prune & 0.932 & 0.949 & 0.916 & 0.580 & 0.589 & 0.570 \\
Omission + Prune & 0.935 & 0.940 & 0.931 & 0.574 & 0.530 & 0.626 \\
Omission + Prune + Evidence & 0.935 & 0.942 & 0.928 & 0.590 & 0.570 & 0.612 \\
\bottomrule
\end{tabular}

